# 2.9. Poszukiwanie najlepszej konfiguracji drzewa
Stworzyliśmy pewien bazowy model klasyfikacji oparty o drzewa decyzyjne, ale skorzystaliśmy wyłącznie z domyślnych parametrów biblioteki scikit-learn. Gdybyśmy na tej podstawie odrzucili cały algorytm jako nieskuteczny, to prawdopodobnie powinniśmy podarować sobie Machine Learning i zająć się inną dziedziną. W oczywisty sposób algorytm może okazać się skuteczny dopiero gdy poprawnie go skonfigurujemy, a do procesu doboru optymalnych wartości warto skorzystać z walidacji krzyżowej, żeby nasz wynik nie był wynikiem ślepego trafu.

In [1]:
import pandas as pd
import numpy as np

In [4]:
purchases_df = pd.read_parquet("../data/purchases_df.parquet")
purchases_df.sample(5)

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,Month,OperatingSystems,Browser,Region,TrafficType,VisitorType,Weekend,Revenue
8377,5,98.750000,0,0.0,126,6085.002381,0.000000,0.004264,28.798471,0.0,Nov,2,2,1,2,Returning_Visitor,0,True
5150,6,46.272727,0,0.0,113,2216.668320,0.005357,0.014345,8.353508,0.2,May,2,2,1,1,Returning_Visitor,0,False
1005,0,0.000000,0,0.0,15,1674.000000,0.000000,0.040000,0.000000,0.0,Mar,2,2,4,1,Returning_Visitor,0,False
7714,5,125.100000,2,34.6,25,887.750000,0.013793,0.023755,0.000000,0.0,Aug,2,2,1,1,Returning_Visitor,1,False
6302,0,0.000000,1,297.2,8,320.266667,0.000000,0.044444,17.795282,0.0,Oct,2,2,1,20,Returning_Visitor,0,True


In [5]:
purchases_df = purchases_df.astype({
    "OperatingSystems": "category",
    "Browser": "category",
    "Region": "category",
    "TrafficType": "category",
    "Weekend": "category",
})
purchases_df.dtypes

Administrative                int64
Administrative_Duration     float64
Informational                 int64
Informational_Duration      float64
ProductRelated                int64
ProductRelated_Duration     float64
BounceRates                 float64
ExitRates                   float64
PageValues                  float64
SpecialDay                  float64
Month                        object
OperatingSystems           category
Browser                    category
Region                     category
TrafficType                category
VisitorType                  object
Weekend                    category
Revenue                        bool
dtype: object

In [6]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

In [7]:
categorical_features = purchases_df.select_dtypes(include=['object', 'category']).columns
categorical_features

Index(['Month', 'OperatingSystems', 'Browser', 'Region', 'TrafficType',
       'VisitorType', 'Weekend'],
      dtype='object')

In [8]:
categorical_transformer = Pipeline(steps=[
    ("one_hot_encoder", OneHotEncoder(handle_unknown='ignore')),
])

In [9]:
numerical_features = purchases_df.select_dtypes([int, float]).columns
numerical_features

Index(['Administrative', 'Administrative_Duration', 'Informational',
       'Informational_Duration', 'ProductRelated', 'ProductRelated_Duration',
       'BounceRates', 'ExitRates', 'PageValues', 'SpecialDay'],
      dtype='object')

In [11]:
preprocessor = ColumnTransformer(transformers=[
    ("categorical", categorical_transformer, categorical_features),
    ("numerical", "passthrough", numerical_features),
])

In [12]:
X = purchases_df.drop(columns="Revenue")
y = purchases_df["Revenue"]

In [13]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier

In [14]:
param_grid = {
    "criterion": ["gini", "entropy"],
    "splitter": ["best", "random"],
    "max_depth": range(3, 50, 4),
    "min_samples_split": [2, 4, 8, 16],
    "min_samples_leaf": [1, 2, 4, 8],
    "max_features": [None, "sqrt", "log2"],
    "class_weight": [None, "balanced"],
}

In [17]:
classifier = Pipeline(steps=[
    ("preprocessing", preprocessor),
    ("decision_tree", GridSearchCV(DecisionTreeClassifier(random_state=253), param_grid, scoring="f1", n_jobs=6, verbose=1, cv=5, return_train_score=True)),
])

In [18]:
classifier.fit(X, y)

Fitting 5 folds for each of 4608 candidates, totalling 23040 fits


Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('categorical',
                                                  Pipeline(steps=[('one_hot_encoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  Index(['Month', 'OperatingSystems', 'Browser', 'Region', 'TrafficType',
       'VisitorType', 'Weekend'],
      dtype='object')),
                                                 ('numerical', 'passthrough',
                                                  Index(['Administrative', 'Administrative_Duration', 'Inform...
                 GridSearchCV(cv=5,
                              estimator=DecisionTreeClassifier(random_state=253),
                              n_jobs=6,
                              param_grid={'class_weight': [None, 'balanced'],
                                          'criterion': ['gini', 'entropy'],
                                          'max_depth': range(3, 50, 4),
                                          'max_features': [None, 'sqrt',
                                                           'log2'],
                                          'min_samples_leaf': [1, 2, 4, 8],
                                          'min_samples_split': [2, 4, 8, 16],
                                          'splitter': ['best', 'random']},
                              return_train_score=True, scoring='f1',
                              verbose=1))])

### Osiągnięte rezultaty
Algorytm uczenia drzewa przejrzał właśnie 4608 różnych konfiguracji i wybrał tę, która maksymalizuje wartość F1. Zobaczmy jak wyglądają wyniki oraz jakiego rodzaju parametry były najlepsze w toku eksperymentowania.

In [19]:
classifier.named_steps["decision_tree"].best_estimator_

DecisionTreeClassifier(class_weight='balanced', criterion='entropy',
                       max_depth=3, max_features='sqrt', random_state=253)

In [20]:
classifier.named_steps["decision_tree"].best_score_

0.6639977183036221

Możemy bez problemu dobrać się do wartości naszej metryki dla każdej przetestowanej konfiguracji, ale że testowaliśmy ich aż tak dużo, to zobaczmy tylko jak prezentowały się wyniki dla tej najlepszej.

In [21]:
results_df = pd.DataFrame(classifier.named_steps["decision_tree"].cv_results_)

In [22]:
results_df.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_class_weight,param_criterion,param_max_depth,param_max_features,param_min_samples_leaf,param_min_samples_split,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
0,0.024723,0.001072,0.003577,0.000278,None,gini,3,None,1,2,...,0.592988,0.078309,529,0.634113,0.658047,0.659493,0.653458,0.666432,0.654309,0.010922
1,0.013161,0.000644,0.003255,0.000137,None,gini,3,None,1,2,...,0.076757,0.140760,4512,0.001309,0.248344,0.024469,0.005222,0.005222,0.056913,0.096057
2,0.026795,0.005885,0.003636,0.000576,None,gini,3,None,1,4,...,0.592988,0.078309,529,0.634113,0.658047,0.659493,0.653458,0.666432,0.654309,0.010922
3,0.013003,0.000970,0.003724,0.000466,None,gini,3,None,1,4,...,0.076757,0.140760,4512,0.001309,0.248207,0.024469,0.005222,0.005222,0.056886,0.096002
4,0.024854,0.002590,0.003467,0.000402,None,gini,3,None,1,8,...,0.592988,0.078309,529,0.634113,0.658047,0.659493,0.653458,0.666432,0.654309,0.010922


In [23]:
idx = classifier.named_steps["decision_tree"].best_index_

In [24]:
idx

3488

In [25]:
results_df.iloc[idx]

mean_fit_time                                                       0.007596
std_fit_time                                                        0.000601
mean_score_time                                                      0.00317
std_score_time                                                      0.000337
param_class_weight                                                  balanced
param_criterion                                                      entropy
param_max_depth                                                            3
param_max_features                                                      sqrt
param_min_samples_leaf                                                     1
param_min_samples_split                                                    2
param_splitter                                                          best
params                     {'class_weight': 'balanced', 'criterion': 'ent...
split0_test_score                                                   0.849498

In [26]:
best_classifier = classifier.named_steps["decision_tree"].best_estimator_

In [28]:
best_classifier

DecisionTreeClassifier(class_weight='balanced', criterion='entropy',
                       max_depth=3, max_features='sqrt', random_state=253)

### Możliwe usprawnienia
W Tym przykładzie wykorzystaliśmy zbiór danych w dokładnie taki sposób, w jaki on został zapisany i nie przeprowadziliśmy żadnych zawansowanych obliczeń by go usprawnić. Posiadamy cechy, które powinny chyba zostać zapisane w inny sposób, tak aby zachować swoją cykliczność - mowa oczywiście o miesiącu. Styczeń jest bliżej grudnia niż czerwca ale przez potraktowanie tej cechy tak, jakby była kategorią, utraciliśmy tę informację.

Ciekawe wydaje się także być wykorzystanie cech pochodnych od liczby odwiedzonych stron danego typu oraz czasu spędzonego na danego rodzaju stronach. Logiczne jest, że suma czasu będzie rosła wraz ze wzrostem liczby odwiedzonych stron, ale może średni czas odwiedzin będzie istotniejszy?

Na początek zobaczmy wagi jakie model przypisał do poszczególnych cech.

In [29]:
best_params = results_df.iloc[idx]["params"]
best_params

{'class_weight': 'balanced',
 'criterion': 'entropy',
 'max_depth': 3,
 'max_features': 'sqrt',
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'splitter': 'best'}

In [31]:
one_hot_features = classifier.named_steps["preprocessing"].named_transformers_["categorical"].named_steps["one_hot_encoder"].get_feature_names_out(categorical_features)
final_features = list(one_hot_features) + list(numerical_features)
results_df = pd.DataFrame({"importance": best_classifier.feature_importances_},
                          index=final_features)
results_df.nlargest(n=10, columns="importance")

,importance
PageValues,0.907759
ProductRelated,0.042428
SpecialDay,0.025485
Administrative_Duration,0.016847
TrafficType_4,0.006436
Informational,0.001045
Month_Aug,0.000000
Month_Dec,0.000000
Month_Feb,0.000000
Month_Jul,0.000000


Część cech ma wagi zerowe, więc model nie odnalazł w nich żadnej wartości. Wagi poszczególnych cech mają różne rzędy wielkości, więc może część z nich to tylko szum, który przez przypadek wykazał jakieś własności charakterystyczne dla klas. Warto to sprawdzić i np. zastosować mechanizn rekursywnej eliminacji cech, jak w poprzednim przypadku. Znamy najlepiej jednak obiecującą konfigurację i teraz tylko zbadamy jaka jest skuteczność modelu gdy pozbędziemy się części kolumn.

In [32]:
from sklearn.feature_selection import RFECV

In [34]:
best_configuration_tree = DecisionTreeClassifier(random_state=253, **best_params)

rfecv = RFECV(best_configuration_tree, cv=5, scoring="f1")
rfecv_classifier = Pipeline(steps=[
    ("preprocessing", preprocessor),
    ("feature_elimination", rfecv),
    ("decision_tree", best_configuration_tree)
])

In [35]:
rfecv_classifier.fit(X, y)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('categorical',
                                                  Pipeline(steps=[('one_hot_encoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  Index(['Month', 'OperatingSystems', 'Browser', 'Region', 'TrafficType',
       'VisitorType', 'Weekend'],
      dtype='object')),
                                                 ('numerical', 'passthrough',
                                                  Index(['Administrative', 'Administrative_Duration', 'Inform...
       'BounceRates', 'ExitRates', 'PageValues', 'SpecialDay'],
      dtype='object'))])),
                ('feature_elimination',
                 RFECV(cv=5,
                       estimator=DecisionTreeClassifier(class_weight='balanced',
                                                        criterion='entropy',
                                                        max_depth=3,
                                                        max_features='sqrt',
                                                        random_state=253),
                       scoring='f1')),
                ('decision_tree',
                 DecisionTreeClassifier(class_weight='balanced',
                                        criterion='entropy', max_depth=3,
                                        max_features='sqrt',
                                        random_state=253))])

In [38]:
np.max(rfecv_classifier.named_steps["feature_elimination"].cv_results_["mean_test_score"])

0.6639977183036221

In [39]:
rfecv_classifier.named_steps["feature_elimination"].n_features_

3

In [40]:
rfe_best_classifier = rfecv.estimator_

In [42]:
results_df = pd.DataFrame({"selected": rfecv.support_},
                          index=final_features)
results_df.query("selected == True")

,selected
ProductRelated_Duration,True
BounceRates,True
PageValues,True


W tym przypadku okazało się, że zastosowanie rekursywnej eliminacji cech nie wystarcza, aby zwiększyć skuteczność modelu. Spróbujemy jeszcze trochę poprawić nasze aktualne osiągi za pomocą innych algorytmów opartych o drzewa decyzyjne. W następnym rozdziale postaramy się trochę lepiej zrozumieć sposób wnioskowania modelu, ale zapiszmy cały łańcuch przetwarzania, żeby nie musieć nauczać go ponownie.

In [43]:
import joblib

In [50]:
rfecv_classifier

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('categorical',
                                                  Pipeline(steps=[('one_hot_encoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  Index(['Month', 'OperatingSystems', 'Browser', 'Region', 'TrafficType',
       'VisitorType', 'Weekend'],
      dtype='object')),
                                                 ('numerical', 'passthrough',
                                                  Index(['Administrative', 'Administrative_Duration', 'Inform...
       'BounceRates', 'ExitRates', 'PageValues', 'SpecialDay'],
      dtype='object'))])),
                ('feature_elimination',
                 RFECV(cv=5,
                       estimator=DecisionTreeClassifier(class_weight='balanced',
                                                        criterion='entropy',
                                                        max_depth=3,
                                                        max_features='sqrt',
                                                        random_state=253),
                       scoring='f1')),
                ('decision_tree',
                 DecisionTreeClassifier(class_weight='balanced',
                                        criterion='entropy', max_depth=3,
                                        max_features='sqrt',
                                        random_state=253))])

In [51]:
joblib.dump(rfecv_classifier, "../model/purchases_dtree.joblib")

['../model/purchases_dtree.joblib']

In [45]:
import json

In [49]:

with open("../model/purchases_dtree.features", 'w') as fp:
    json.dump(final_features, fp)